In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from pandas import read_csv
from pandas import concat
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import plot_model

In [3]:
ps_files = os.listdir(os.path.join(os.getcwd(),'Datasets','Abnormal'))
ps_files

['abnormal01_aaaaabdo_s003_t000.edf.csv',
 'abnormal02_aaaaaddm_s006_t000.edf.csv',
 'abnormal03_aaaaadeu_s002_t000.edf.csv',
 'abnormal04_aaaaadkb_s002_t000.edf.csv',
 'abnormal06_aaaaagsc_s006_t001.edf.csv',
 'abnormal07_aaaaagvx_s002_t000.edf.csv',
 'abnormal08_aaaaagvx_s003_t001.edf.csv',
 'abnormal_aaaaabuv_s002_t000.edf.csv',
 'abnornal05_aaaaagsc_s003_t001.edf.csv']

In [4]:
ng_files = os.listdir(os.path.join(os.getcwd(),'Datasets','Normal'))
ng_files

['normal01_aaaaacad_s003_t000.edf.csv',
 'normal02_aaaaacby_s004_t000.edf.csv',
 'normal03_aaaaadjk_s002_t000.edf.csv',
 'normal04_aaaaadsm_s002_t001.edf.csv',
 'normal_aaaaaayx_s002_t000.edf.csv']

In [5]:
def import_data(files, state):
#Import dataframe
    cols = [I for I in range(32)]
    datasets = []
    for file in files:
        ps_df = read_csv(os.path.join(os.getcwd(),'Datasets', str(state),file))
        ps_df = ps_df[ps_df.columns[:32]]
        if state.lower() == 'abnormal':
            ps_df['label'] = np.ones(ps_df[ps_df.columns[0]].shape)
        elif state.lower() == 'normal':
            ps_df['label'] = np.zeros(ps_df[ps_df.columns[0]].shape)
        try:
            ps_df.columns==cols
            print(file)
            # Replace these with your actual data
            if ps_df.shape[1]==33:
                X_train_dataset = ps_df.drop('label',axis=1)
                Y_train_dataset =  ps_df['label']
                datasets.append((X_train_dataset, Y_train_dataset))
        except Exception as e:
            pass
        cols = ps_df.columns
    return datasets

In [6]:
ps = import_data(ps_files, 'Abnormal')
ng = import_data(ng_files, 'Normal')

abnormal02_aaaaaddm_s006_t000.edf.csv
abnormal04_aaaaadkb_s002_t000.edf.csv
abnormal06_aaaaagsc_s006_t001.edf.csv
abnormal07_aaaaagvx_s002_t000.edf.csv
abnornal05_aaaaagsc_s003_t001.edf.csv
normal02_aaaaacby_s004_t000.edf.csv
normal03_aaaaadjk_s002_t000.edf.csv
normal04_aaaaadsm_s002_t001.edf.csv
normal_aaaaaayx_s002_t000.edf.csv


In [7]:
datasets = []
len(ps), len(ng)

(4, 4)

In [8]:
for p in ps:
    print(len(p))
    datasets.append(p)

2
2
2
2


In [9]:
for n in ng:
    print(len(n))
    datasets.append(n)
len(datasets)

2
2
2
2


8

In [10]:
# Combine all datasets into a single dataset
X_train_combined = np.concatenate([dataset[0] for dataset in datasets], axis=0)
Y_train_combined = np.concatenate([dataset[1] for dataset in datasets], axis=0)
X_train_combined.shape, Y_train_combined.shape

((2520750, 32), (2520750,))

In [11]:
# Normalize the features
scaler = StandardScaler()
X_train_combined = scaler.fit_transform(X_train_combined)

In [12]:
test_df = read_csv("test1.edf.csv")
test_df = test_df[test_df.columns[:32]]
test_df['label'] = np.ones(test_df[test_df.columns[0]].shape)
test_df.head()
test_df.columns

Index(['# EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF',
       'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF',
       'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF',
       'EEG T5-REF', 'EEG T6-REF', 'EEG A1-REF', 'EEG A2-REF', 'EEG FZ-REF',
       'EEG CZ-REF', 'EEG PZ-REF', 'EEG ROC-REF', 'EEG LOC-REF',
       'EEG EKG1-REF', 'EMG-REF', 'EEG 26-REF', 'EEG 27-REF', 'EEG 28-REF',
       'EEG 29-REF', 'EEG 30-REF', 'EEG T1-REF', 'EEG T2-REF', 'label'],
      dtype='object')

In [13]:
len(test_df.columns)


33

In [14]:
# Define a function to create the DNN model
def create_model():
    model = Sequential()
    # Modify the input dimension of the first layer to match the number of features in your input data (32 in this case)
    # 30000
    model.add(Dense(300, input_dim=32, activation='relu'))
    # 15000
    model.add(Dense(150, activation='relu'))
    model.add(Dropout(0.5))
    # 7500
    model.add(Dense(75, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model

In [15]:
# Create and compile the model for dataset 1
model_dataset = create_model()
model_dataset.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
X_train_combined.shape, Y_train_combined.shape

((2520750, 32), (2520750,))

In [17]:
# Convert the labels into one-hot encoding
Y_train1_onehot = tf.keras.utils.to_categorical(Y_train_combined, num_classes=2)

In [18]:
# Split dataset 1 into train and validation sets
X_train1, X_val1, Y_train1, Y_val1 = train_test_split(X_train_combined, Y_train1_onehot, test_size=0.2, random_state=42)

In [19]:
X_test1 = test_df.drop('label',axis=1)
Y_test1 = test_df['label']

X_val, X_test, Y_val, Y_test = train_test_split(X_test1, Y_test1, test_size=0.5, random_state=42)

# Convert the test labels into one-hot encoding
Y_test_onehot = tf.keras.utils.to_categorical(Y_test, num_classes=2)

X_val1.shape, Y_val1.shape

((504150, 32), (504150, 2))

In [20]:
# Train the model on dataset 1
model_dataset.fit(X_train1, Y_train1, epochs=3, batch_size=128, validation_data=(X_val1, Y_val1))

Epoch 1/3
15755/15755 [==============================] - 54s 3ms/step - loss: 0.2486 - accuracy: 0.8960 - val_loss: 0.1554 - val_accuracy: 0.9378
Epoch 2/3
15755/15755 [==============================] - 49s 3ms/step - loss: 0.1612 - accuracy: 0.9374 - val_loss: 0.1293 - val_accuracy: 0.9489
Epoch 3/3
15755/15755 [==============================] - 51s 3ms/step - loss: 0.1401 - accuracy: 0.9463 - val_loss: 0.1180 - val_accuracy: 0.9536


In [21]:
# Evaluate the model on dataset 1
loss, accuracy = model_dataset.evaluate(X_test, Y_test_onehot)
print(f"Dataset 1 Validation Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

4704/4704 [==============================] - 7s 1ms/step - loss: 0.1338 - accuracy: 1.0000
Dataset 1 Validation Loss: 0.1338, Accuracy: 1.0000


In [22]:
pred = model_dataset.predict(X_test)
print(pred)

4704/4704 [==============================] - 6s 1ms/step
[[0.02009058 0.9799095 ]
 [0.02009208 0.979908  ]
 [0.0200946  0.9799054 ]
 ...
 [0.02005781 0.9799422 ]
 [0.02009596 0.97990406]
 [0.02008194 0.97991806]]


In [24]:
# Plot the model architecture
plot_model(model_dataset, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
